In [25]:
import os, random, sys, time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from math import pi

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException

from parsel import Selector

In [31]:
#import Excel file from Part 1

linkedin_links = pd.read_excel('/Users/vpaskannaya/Talent Analytics Live/linkedin_links.xls', sheet_name='Sheet1', header=None)
#uncomment to check the result
linkedin_links.tail()

,0
647,https://www.linkedin.com/in/normand-bessette-1...
648,https://www.linkedin.com/in/jazzbhath?miniProf...
649,https://www.linkedin.com/in/fabrice-riva-60bb4...
650,https://www.linkedin.com/in/vbusnita?miniProfi...
651,https://www.linkedin.com/in/victoriapaskannaya...


In [28]:
#create a new table and populate with expected headers
columns=['url', 'profile_name','job_title','duration','company1','company2']
df = pd.DataFrame(columns=columns)
#uncomment to check if it is emprty
#df.head()

In [29]:
#Jeff back at work!
browser = webdriver.Chrome('/Users/vpaskannaya/Talent Analytics Live/chromedriver')

browser.get('https://www.linkedin.com/uas/login')

file = open('Config.txt')
lines = file.readlines()
username = lines[0]
password = lines[1]

elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password)

elementID.submit()

n=0
while n < len(linkedin_links):
    profilelink = linkedin_links[0][n]
    browser.get(profilelink)
    
    time.sleep(2)
    
    page=BeautifulSoup(browser.page_source)

    profile_name=page.find('h1',{'class':'text-heading-xlarge inline t-24 v-align-middle break-words'}).get_text(strip=True)

    job_title = page.find('div',{'class':'text-body-medium break-words'}).get_text(strip=True)
    
    try:
        duration = page.find('span',{'class':'pv-entity__bullet-item-v2'}).get_text(strip=True)
    except:
        duration = None

    previous_employers = page.find('section',{'id':'experience-section'})
    
    time.sleep(2)
    try:
        company1=previous_employers.findAll('p',{'class':'pv-entity__secondary-title'})[1].get_text('|',strip=True)
        company1=company1[:company1.rfind('|')]
    except:
        company1=None

    try:
        company2=previous_employers.findAll('p',{'class':'pv-entity__secondary-title'})[2].get_text('|',strip=True)
        company2=company2[:company2.rfind('|')]
    except:
        print(linkedin_links[0][n])
        company2=None
    #append each result to your table
    df=df.append({'url' : profilelink, 'profile_name' : profile_name,'duration' : duration,'job_title' : job_title,'company1' : company1,'company2' : company2},ignore_index=True)
    
    n=n+1

https://www.linkedin.com/in/marianne-roberge-772b7820a?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADU11UYB0qOsRLtRETVXzDd8SWICVufhA8Q
https://www.linkedin.com/in/rosalie-liu?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACDv-sEB0Qan0-D_RLUEOQtBQX4DW7m48Rk
https://www.linkedin.com/in/louis-bourdages-91521647?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAnSBqEBrQfEnsyONS0P89DwygxkS7uoVc8
https://www.linkedin.com/in/bobby-avoine-52892418a?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACyhi_ABPy2DIlCcyBWpf2vYYpDclwpRVWE
https://www.linkedin.com/in/marilyne-chan-06541024?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAULQU0B6YV170BYVvZXea00PyZYOQC-3x0
https://www.linkedin.com/in/francis-piche?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACCyBr0BYy-DIHwaUBoFUEdyZqFckbs1g0A
https://www.linkedin.com/in/nathan-lafrance-berger-394771120?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAB4DU-ABmtk_7MEDCNKoal2qxA6cNXICw7w
https://www.linkedin.com/in/daniel-lavoie-9023aa5?miniProfileUrn=u

AttributeError: 'NoneType' object has no attribute 'get_text'

In [42]:
#uncomment to see the resulting table
df.tail()
#export this table to excel
#df.to_excel('/Users/vpaskannaya/Talent Analytics Live/export_linkedin_profiles.xls', index = False, header=True)

,url,profile_name,job_title,duration,company1,company2
359,https://www.linkedin.com/in/patrick-john-chia-...,Patrick John Chia,Machine Learning Developer at Coveo,None,None,None
360,https://www.linkedin.com/in/gabriel-ruel-fiset...,Gabriel Ruel-Fiset,Security Analyst,None,None,None
361,https://www.linkedin.com/in/martin-gravel-aa6b...,Martin Gravel,Développeur R&D chez Coveo,None,None,None
362,https://www.linkedin.com/in/sophie-baillargeon...,Sophie Baillargeon-Laporte,"Software Engineering Intern, R&D at Coveo",None,None,None
363,https://www.linkedin.com/in/denis-blanchette-5...,Denis Blanchette,Team Leader chez Coveo,None,None,None


In [53]:
from tinydb import TinyDB

db = TinyDB('/Users/vpaskannaya/Talent Analytics Live/databases/xport_linkedin_profiles_db.json')

i=0
while i < len(df):
    db.insert({'url': df['url'][i], 'profile_name': df['profile_name'][i], 'job_title':df['job_title'][i], 'duration':df['duration'][i], 'company1':df['company1'][i], 'company2':df['company2'][i]})
    i=i+1